### Setup LLM using Ollama

In [1]:
import nest_asyncio

nest_asyncio.apply()

In [2]:
from llama_index.llms.ollama import Ollama

llm = Ollama(model="llama3", request_timeout=120.0)

/Users/linghuang/miniconda3/envs/llama3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# Call complete with a prompt
response = llm.complete("do you like drake or kendrick better?")

print(response)

I'm just an AI, I don't have personal preferences or opinions, nor do I have the ability to listen to music or experience emotions. However, I can provide information about both Drake and Kendrick Lamar, two highly acclaimed and influential rappers.

Drake is a Canadian rapper, singer, songwriter, and record producer who has been widely successful with his blend of hip-hop, R&B, and pop. He's known for his introspective lyrics, melodic flow, and catchy hooks. Some of his most popular songs include "God's Plan," "One Dance," and "In My Feelings."

Kendrick Lamar is an American rapper, singer, songwriter, and record producer who has gained widespread recognition for his socially conscious and lyrically dense music. He's known for his storytelling ability, jazz-infused production, and powerful messages. Some of his most popular songs include "Alright," "Humble," and "DNA."

Both artists have their unique strengths and styles, and opinions about which one is better often come down to perso

In [6]:
stream_response = llm.stream_complete(
    "you're a drake fan. tell me why you like drake more than kendrick"
)

for t in stream_response:
    print(t.delta, end="")

I'm a Drake fan, and I gotta give you my honest opinion!

While Kendrick Lamar is an incredible artist, I have to say that Drake's music resonates with me on a deeper level. Here are some reasons why:

1. **Authenticity**: Drake's lyrics are often deeply personal and relatable, drawing from his own experiences growing up in Toronto as a black man. He doesn't shy away from exploring themes of vulnerability, self-doubt, and the struggles of fame. For me, this authenticity makes his music more impactful and genuine.
2. **Versatility**: Drake has demonstrated an impressive range across multiple genres – hip-hop, R&B, pop, and even electronic-infused tracks. His ability to experiment and adapt to different styles keeps his sound fresh and exciting. Kendrick, on the other hand, tends to focus more on socially conscious hip-hop with a strong emphasis on storytelling.
3. **Melodic mastery**: Drake's melodic sensibilities are unparalleled in modern hip-hop. He has an incredible ear for crafting

In [7]:
from llama_index.core.llms import ChatMessage

# Call chat with a list of messages
messages = [
    ChatMessage(role="system", content="You are Kendrick."),
    ChatMessage(role="user", content="Write a verse."),
]
response = llm.chat(messages)

In [8]:
print(response)

assistant: "Look at my life, it's a movie, feel me?
I'm the king of the game, and I'm just gettin' started, G
From Compton to the world, I made my mark, yeah
My rhymes so hot, they'll leave you scarred"


## Basic RAG (Vector Search, Summarization)

In [9]:
from dotenv import load_dotenv, find_dotenv
import os

In [10]:
_ = load_dotenv(find_dotenv())

In [11]:
import pandas as pd

In [14]:
from datasets import load_dataset

In [15]:
xsum_dataset = load_dataset(
    "xsum", version="1.2.0"
)

/Users/linghuang/miniconda3/envs/llama3/lib/python3.10/site-packages/datasets/load.py:1486: FutureWarning: The repository for xsum contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/xsum
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [16]:
# Taking a sample of 1000 rows
xsum_sample = xsum_dataset["train"].select(range(1000)).to_pandas()
xsum_sample.head(2)

,document,summary,id
0,"The full cost of damage in Newton Stewart, one...",Clean-up operations are continuing across the ...,35232142
1,A fire alarm went off at the Holiday Inn in Ho...,Two tourist buses have been destroyed by fire ...,40143035


In [17]:
xsum_sample["combined"] = (
    "Document: " + xsum_sample.document.str.strip() + "; Summary: " + xsum_sample.summary.str.strip()
)

In [18]:
!mkdir -p 'document/'
documents = xsum_dataset["train"].select(range(1000)).to_pandas()
joined_documents = '\n'.join(xsum_sample["combined"])
with open('document/documents.txt', 'w', encoding='utf-8') as file:
    file.write(joined_documents)

In [19]:
from llama_index.core import SimpleDirectoryReader

loader = SimpleDirectoryReader(input_dir="./document/")
documents = loader.load_data()

In [20]:
from llama_index.core import VectorStoreIndex

In [21]:
index = VectorStoreIndex.from_documents(documents)

In [22]:
query_engine = index.as_query_engine(similarity_top_k=3)

In [23]:
response = query_engine.query("Tell me about Harry Potter")

In [24]:
print(str(response))

The play, written by Jack Thorne, is set 19 years after the seventh and final book in the series by JK Rowling. It opens officially at the Palace Theatre, in London's West End, on Saturday. Audiences have been urged to "keep the secrets" since the play began previews in early June. Presented in two parts, the play - showing the stars of the wizarding saga as adults in their mid-30s as their own children head off to school - stretches over five hours.


### Basic RAG (Summarization)

In [25]:
from llama_index.core import SummaryIndex

summary_index = SummaryIndex.from_documents(documents)
summary_engine = summary_index.as_query_engine()

In [26]:
response = summary_engine.query(
    "Given your assessment of these documents, which news impressed you most?"
)

ReadTimeout: timed out

In [ ]:
print(str(response))

In [3]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

/Users/linghuang/miniconda3/envs/llama3/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [4]:
from llama_index.core import Settings

Settings.llm = llm
Settings.embed_model = embed_model

In [4]:
import ollama
response = ollama.chat(model='llama3', messages=[
  {
    'role': 'user',
    'content': 'Why is the sky blue?',
  },
])
print(response['message']['content'])

The sky appears blue to our eyes because of a phenomenon called Rayleigh scattering. This is when shorter, blue wavelengths of light are scattered more than longer, red wavelengths by the tiny molecules of gases in the atmosphere, such as nitrogen and oxygen.

Here's what happens:

1. Sunlight enters Earth's atmosphere and contains all the colors of the visible spectrum (red, orange, yellow, green, blue, indigo, and violet).
2. As sunlight travels through the atmosphere, it encounters tiny molecules of gases like nitrogen (N2) and oxygen (O2). These molecules are much smaller than the wavelength of light.
3. When a photon of light hits one of these molecules, it scatters in all directions due to the molecule's small size compared to the wavelength of light.
4. The amount of scattering that occurs depends on the wavelength of the light. Shorter wavelengths (like blue and violet) are scattered more than longer wavelengths (like red and orange). This is because the smaller molecules are b

In [5]:
import os
import openai
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv()) # read local .env file

In [6]:
hf_token = os.getenv("HF_API_KEY")

In [7]:
import transformers
import torch

model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

pipeline = transformers.pipeline(
  "text-generation",
  model="meta-llama/Meta-Llama-3-8B-Instruct",
  model_kwargs={"torch_dtype": torch.bfloat16},
  device="cuda",
)

/Users/linghuang/miniconda3/envs/llama3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [01:01<00:00, 15.39s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


AssertionError: Torch not compiled with CUDA enabled

In [3]:
llm = Ollama(model="llama3", request_timeout=120.0)

NameError: name 'Ollama' is not defined

In [8]:
resp = llm.complete("Who is Paul Graham?")

In [9]:
print(resp)

Paul Graham (1919-2011) was a British Methodist minister and civil rights activist. He was a key figure in the Northern Ireland civil rights movement of the 1960s, which aimed to end discrimination against Catholics and promote equal rights for all citizens.

Graham was born in Belfast, Northern Ireland, and grew up in a Protestant family. However, he became increasingly involved with the Methodist Church and its social justice concerns, leading him to become a vocal advocate for Catholic civil rights. He believed that the Protestant-dominated government and society were unfairly denying Catholics equal access to education, employment, and housing.

In 1967, Graham was one of the founding members of the Northern Ireland Civil Rights Association (NICRA), which aimed to use peaceful protest and legal means to challenge discrimination. He became a prominent figure in the movement, often appearing alongside other civil rights leaders like Ivan Cooper and Eamon McCann.

Graham's activism wa

In [11]:
import ollama
response = ollama.chat(model='llama3', messages=[
  {
    'role': 'user',
    'content': 'Why is the sky blue?',
  },
])
print(response['message']['content'])

ModuleNotFoundError: No module named 'ollama'